In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pytensor.tensor as pt
import polars as pl
print(f"Running on PyMC v{pm.__version__}")
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")


Running on PyMC v5.10.4


In [2]:
##READ IN
df=pl.read_csv('goalie_df.csv', null_values = 'NA')

In [11]:
df.head()

,event_type,xg,secondary_type,event_goalie_id,goal,goalie_id_stan
i64,str,f64,str,i64,i64,i64
7,"""SHOT""",-0.367581,"""wrist""",8477424,0,33
8,"""SHOT""",0.914102,"""tip-in""",8477424,0,33
10,"""SHOT""",-0.387941,"""snap""",8477424,0,33
16,"""SHOT""",-0.384078,"""wrist""",8477424,0,33
28,"""SHOT""",-0.349418,"""snap""",8477992,0,41


In [3]:
#convert shot type to one hot encoding
shot_df = df.select('secondary_type').to_dummies()

In [4]:
#add intercept
ones_df = pl.DataFrame({"ones": [1] * df.height})
shot_df = pl.concat([ones_df, shot_df], how = 'horizontal')

In [25]:
shot_df.head()

ones,secondary_type_backhand,secondary_type_bat,secondary_type_between-legs,secondary_type_cradle,secondary_type_deflected,secondary_type_poke,secondary_type_slap,secondary_type_snap,secondary_type_tip-in,secondary_type_wrap-around,secondary_type_wrist
i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
1,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1,0,0,0


In [27]:
shot_df.width

12

In [5]:
# Data
J = df['goalie_id_stan'].n_unique()
N = df.height  # number of shots
B = shot_df.width  # number of predictors for the hierarchical prior on beta (including intercept)
S = np.array(shot_df)  # matrix of shot characteristics for hierarchical prior on beta
jj = df['goalie_id_stan']  # goalie for observation n
y = df['goal']  # correctness for observation n



In [7]:
with pm.Model() as irt_model:
    #priors
    #shot difficulty
    b = pm.Normal('b', mu = 0, sigma = 1, shape = B)
    sigma_b = pm.HalfCauchy('sigma_b', beta = 1)
    mu_beta = pm.Normal('mu_beta', mu=pm.math.dot(S,b), sigma=sigma_b, shape=N)

    #priors on irt components
    alpha = pm.Normal('alpha', mu = 0, sigma = 1, shape = J)
    sigma_beta = pm.HalfCauchy('sigma_beta', beta=1)
    beta = pm.Normal('beta', mu = mu_beta, sigma = sigma_beta)
    delta = pm.Normal('delta', mu=0.75, sigma=1)

    #likelihood
    for n in range(N):
        p = pm.Deterministic("p{}".format(n),pm.math.sigmoid(alpha[jj[n]] - beta[n] + delta))
        pm.Bernoulli(f'y_{n}', p=p, observed=y[n])

      # Sample
    trace = pm.sample(draws = 500, cores=1)

KeyboardInterrupt: 